<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#教材サイト" data-toc-modified-id="教材サイト-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>教材サイト</a></span><ul class="toc-item"><li><span><a href="#検定手法のリストアップ" data-toc-modified-id="検定手法のリストアップ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>検定手法のリストアップ</a></span></li></ul></li><li><span><a href="#実験" data-toc-modified-id="実験-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>実験</a></span></li><li><span><a href="#カイ２乗検定" data-toc-modified-id="カイ２乗検定-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>カイ２乗検定</a></span></li></ul></div>

# 概要
広告やキャンペーン効果測定の方法についてまとめる。  
繰り返し使いまわせるような体系化された内容にしよう。

# 教材サイト

A/Bテストに用いられれる統計的検定手法（ロジック）のまとめ＆比較 | RCO Ad-Tech Lab Blog  
https://www.rco.recruit.co.jp/career/engineer/blog/ab-test-logic/

仮説検証とサンプルサイズの基礎 - クックパッド開発者ブログ
https://techlife.cookpad.com/entry/2016/09/26/111601


長文であるが、統計的仮説検定の盲点について考察している。  
観測結果を鵜呑みにしないための視点が記されている。
http://id.fnshr.info/2014/12/17/stats-done-wrong-05/


## 検定手法のリストアップ

1. **二項検定**
    クリック数（コンバージョン率）常に使える。
    理論的に厳密。フィッシャーの正確確率検定に対して、”戻しの処理”を加えたもの。復元抽出。
    
2. **カイ二乗検定**  
   クリック数（コンバージョン率）のサンプルサイズが大きい場合（＞O(10)）に使える。
   二項検定において、分布（二項分布）をサンプルサイズが大きい極限において正規分布で近似（中心極限定理）
   
3. **フィッシャーの正確確率検定**
    クリック数（コンバージョン率）に対して常に使える。
    理論的に厳密。二項検定とは違い、抽出したサンプルを戻さないと考える。非復元抽出。
    
4.  **t検定**  
    クリック数（コンバージョン率）や売上等の連続的な数値に使える。  
    クリック数の場合：サンプルサイズが大きいこと、  
    連続数値の場合：分布が正規分布 or サンプルサイズが大きいことが使用条件になる。
    
    クリック数の場合：サンプルサイズが大きい極限において正規分布で近似（中心極限定理）  
    連続数値の場合：平均値に差があるか否かの検定
    
5. **ウィルコクソンの順位和検定**  
   売上等の連続的な数値に使う。サンプルサイズが大きいことが使用条件で、中央値に差があるか否かの検定に使う。
   

# 実験
以下の条件で実験をする。  

* 施策A:改善案
    コンバージョンした=454
    コンバージョンせず=20,933
    
* 施策B：現状維持
    コンバージョンした=189
    コンバージョンせず=10,845

In [2]:
from scipy import stats
from scipy.stats import t

In [8]:
sample_a = [1] * 454 + [0] * 20933
sample_b = [1] * 189 + [0] * 10845

In [9]:
mean_a = np.mean(sample_a)
std_a = np.std(sample_a, ddof=1)

In [14]:
# 各ユーザーごとにコンバージョンする/しないは１票とみなすと equql_val=False とすべき。
t_value, p_value = stats.ttest_ind(sample_a, sample_b, equal_var=False)


In [13]:
print("p値={}".format(p_value))

p値=0.009499486517327413


In [29]:
stats.binom_test(x=454, n=(454+20933), p=(189)/(10845)*1.0, alternative='two-sided')

3.6206495804123572e-05

In [28]:
454/(454+189)*1.0

0.7060653188180405

In [17]:
import scipy.stats
import numpy as np
data = np.matrix([ [ 454, 20933 ], [ 189, 10845 ] ])
p_a = np.sum(data, axis=1).item((0, 0)) / data.sum()
print("p_a=", p_a)
conversion_total = np.sum(data, axis=0).item(0, 0)
print("conversion_total=", conversion_total)
conversion_a  = data.item(0,0)
print("conversion_a=", conversion_a)
# 確率 p_a の基で、n回試行して、x回 (Aは)yesになった
p_value = scipy.stats.binom_test(x=conversion_a, n=conversion_total, p=p_a, alternative="two-sided")
print("p_value=", p_value)

p_a= 0.659665031924
conversion_total= 643
conversion_a= 454
p_value= 0.0125320280817


In [25]:
conversion_a

454

In [26]:
conversion_total

643

In [27]:
p_a

0.65966503192375314

# カイ２乗検定

In [30]:
import scipy.stats
import numpy as np
data = np.matrix([ [ 454, 20933 ], [ 189, 10845 ] ])
chi2, p, ddof, expected = scipy.stats.chi2_contingency( data , correction=False)
msg = "Test Statistic: {}\np-value: {}\nDegrees of Freedom: {}\n"
print( msg.format( chi2, p, ddof ) )
print( expected )

Test Statistic: 6.291035173658824
p-value: 0.012135015711322547
Degrees of Freedom: 1

[[   424.16461553  20962.83538447]
 [   218.83538447  10815.16461553]]
